In [1]:
!pip install  language_tool_python

In [2]:
import pandas as pd
from arrow import now
from language_tool_python import LanguageTool
from sklearn.metrics import confusion_matrix

time_start = now()
corrector = LanguageTool('en-US')


def score(arg: str) -> float:
    arg = arg.replace('\n', ' ')
    arg = ' '.join(arg.split())
    result = 0
#     result += 0.33 if len(arg) < 1798 else 0
#     result += 0.33 if len(arg.split()) < 290 else 0
    result += 1 if arg == corrector.correct(arg) else 0
    return result

original_df = pd.read_csv(filepath_or_buffer='/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
print('loaded original training data')

ai_df = pd.read_csv(filepath_or_buffer='/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
print('loaded first set of generated training data')

gpt_df = pd.read_csv(filepath_or_buffer='/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv')
print('loaded second set of generated training data')

v2_df = pd.read_csv(filepath_or_buffer='/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv').rename(columns={'label': 'generated'})
v2_df['id'] = v2_df['text'].apply(func=lambda x: x[:16].lower().replace(' ', '').replace('"', '').replace('!', '').replace(':', '')[:8])
print('loaded v2 training data')

seven_df = pd.read_csv(filepath_or_buffer='/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv')
print('loaded seven/v1 set of training data')

train_df = v2_df.sample(frac=1).copy()

train_df['score'] = train_df['text'].apply(func=score, )

print('scoring complete in {}'.format(now() - time_start))

print(confusion_matrix(y_true=train_df['generated'].values, y_pred=train_df['score'].values))

train_df.head(n=10)

loaded original training data
loaded first set of generated training data
loaded second set of generated training data
loaded v2 training data
loaded seven/v1 set of training data
scoring complete in 1:01:11.806781
[[13620    92]
 [  991   647]]


,text,generated,id,score
6733,Studying venus is a worthy persuit despite its...,0,studying,0
1407,"In recent years, there has been a growing move...",1,inrecent,0
13681,My phone buzzed with his name appearing at the...,0,myphoneb,0
15043,Limiting car usage has several benefits for bo...,1,limiting,1
12310,Dear Florda state Senator\n\nThere has been qu...,0,dearflor,0
13071,Opinions\n\nShould you have more that one pers...,0,opinions,0
10414,The face on Mars is just a landform. There is ...,0,thefaceo,0
7131,The use of technology improves people's daily ...,0,theuseof,0
14153,"When asking for advice, it is important to tal...",0,whenaski,0
13523,"""Hey Generic_Name, i'm thinking about going to...",0,heygener,0


In [3]:
from plotly.express import strip
strip(data_frame=train_df, x='generated', y='score')

In [4]:
train_df['generated'].value_counts()

generated
0    13712
1     1638
Name: count, dtype: int64

In [5]:
train_df['score'].value_counts()

score
0    14611
1      739
Name: count, dtype: int64

In [6]:
train_df[train_df['generated'] == 1]

,text,generated,id,score
1407,"In recent years, there has been a growing move...",1,inrecent,0
15043,Limiting car usage has several benefits for bo...,1,limiting,1
1942,In the wake of our growing concern for the env...,1,inthewak,0
1471,Advantages of Limiting Car Usage\n\nIn recent ...,1,advantag,0
14777,"I do not have personal beliefs or preferences,...",1,idonotha,0
...,...,...,...,...
1568,Limiting car usage has numerous advantages tha...,1,limiting,0
1810,Support for the Electoral College or Considera...,1,supportf,0
1433,Advantages of Limiting Car Usage\n\nLimiting c...,1,advantag,0
14423,"Dear Mr. Smith,\n\nI am writting to you today ...",1,dearmr.s,0


In [7]:
train_df.shape

(15350, 4)

In [8]:
from plotly.express import histogram
train_df['text length'] = train_df['text'].str.len()
histogram(data_frame=train_df, x='text length', color='generated')

In [9]:
train_df['token count'] = train_df['text'].apply(lambda x: len(x.split()))
histogram(data_frame=train_df, x='token count', color='generated')

In [10]:
ai_df.head()

,id,prompt_id,text,generated
0,d429f032,0,Advantages of Limiting Car Usage \n\nLimiting ...,1
1,1ce279be,0,Advantages of Limiting Car Usage\n\nLimiting c...,1
2,c9595213,0,Limiting car usage has numerous advantages tha...,1
3,f2266d87,0,The passages provided discuss the advantages o...,1
4,eeace4bd,0,Title: The Advantages of Limiting Car Usage\n\...,1
